In [129]:
# module
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import numpy as np

In [130]:
# Util Functions
from enum import Enum

# 파일을 읽어와 pandas의 DataFrame으로 반환해줌
def get_df_from(path: str, sep: str = ','):
    extension = path.split('.')[-1]
    try:
        if extension == "csv" or extension == "txt":
            df = pd.read_csv(path, sep=sep, on_bad_lines='skip')
            if df.empty:
                print("W: DataFrame is empty.")
                return None
            return df
        else:
            print("E: File extension is not supported.")
            return None
    except FileNotFoundError:
        print(f"E: File not found. Check the path: {path}")
        return None
    except pd.errors.ParserError as e:
        print(f"E: Error parsing CSV file: {e}")
        return None
    except Exception as e:
        print(f"E: An unexpected error occured: {e}")
        return None

# 리스트의 평균을 반환.(기본값 0)
def calculate_mean(lst):
    if lst: # 빈 리스트가 아닌 경우에만 평균 계산
        return np.mean(lst)
    return 0 # 빈 리스트인 경우 NaN 반환

# 리스트의 합을 반환.(기본값 0)
def calculate_sum(lst):
    if lst:
        return np.sum(lst)
    return 0


class ChartShape(Enum):
    BAR = 1
    HISTOGRAM = 2
    LINE = 3
    HEATMAP = 4

# Chart 뽑는 함수
def get_chart_from(df: pd.DataFrame, x_col: str, y_col: str, shape: ChartShape = ChartShape.BAR, real_time: bool = False):
    plt.figure(figsize=(10, 6))
    if shape == ChartShape.BAR:
        plt.bar(df[x_col], df[y_col])
    elif shape == ChartShape.HISTOGRAM:
        plt.hist(df[x_col], bins=10)
    elif shape == ChartShape.LINE:
        plt.plot(df[x_col], df[y_col])
    else:
        plt.plot(df[x_col], df[y_col])

    if real_time:
        plt.gca().xaxis.set_major_locator(mdates.MonthLocator()) # 월 단위로 큰 눈금 표시
        plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d')) # 날짜 형식 지정
    plt.xlabel(x_col)
    plt.ylabel(y_col)
    plt.grid(True)
    plt.tight_layout()
    plt.xticks(df[x_col], rotation=45, ha='right')
    plt.show()

def get_chart_from_series(sr: pd.Series, x_col: str, y_col: str, shape: ChartShape = ChartShape.BAR):
    if shape == ChartShape.BAR:
        sr.plot.bar()
    if shape == ChartShape.LINE:
        sr.plot.line()
    if shape == ChartShape.HISTOGRAM:
        sr.plot.hist()
    plt.xlabel(x_col)
    plt.ylabel(y_col)
    plt.show()

def get_integer(question: str):
    try:
        id = int(input(question))
        return id
    except:
        print(f"E: invalid integer")
        return 0

In [131]:
# Function

folder_path = '../public/kmrd-small/'

file_paths = {
    'castings': folder_path + 'castings.csv',
    'countries': folder_path + 'countries.csv',
    'genres': folder_path + 'genres.csv',
    'movies': folder_path + 'movies.txt',
    'peoples': folder_path + 'peoples.txt',
    'rates': folder_path + 'rates.csv',
}

def get_df_strict_from(path: str, sep: str = ','):
    df = get_df_from(path, sep)
    for _ in range(3):
        if df is not None:
            return df
        new_path = input('오류가 발생했습니다. 정확한 경로를 확인해주세요: ')
        new_sep = input('구분자를 입력해주세요: ')
        df = get_df_from(new_path, new_sep)
    exit()

### 분류 평가

- `rate_random_class` 필드를 추가해서 평점을 반올림
- 이를 이용해 분류 평가한다.
  - Confusion Matrix
  - Accuracy
  - Precision
  - Recall
  - F1 Score

In [132]:
dfs = {}
names = ['rates', 'movies']
for name in names:
    path = file_paths[name]
    if name == 'movies' or name == 'peoples':
        dfs[name] = get_df_strict_from(path, '\t')
    else:
        dfs[name] = get_df_strict_from(path)

In [133]:
class MovieDateLoader:
    def __init__(self, name: str):
        path = file_paths[name]
        if name == 'movies' or name == 'peoples':
            self.df =  get_df_strict_from(path, '\t')
        else:
            self.df = get_df_strict_from(path)

        if name == 'rates':
            self.rates_mean = self.df[self.df.notnull()]['rate'].mean()
            self.df['rate'].fillna(self.rates_mean)
        self.df.fillna(0)
    
class RandomRecommender:
    def __init__(self):
        self.dfs = {}
        self.dfs['rates'] = MovieDateLoader('rates')
        self.dfs['movies'] = MovieDateLoader('movies')

    def run(self, cnt):
        rates_df = self.dfs['rates'].df
        movies_df = self.dfs['movies'].df
        rates_mean = self.dfs['rates'].rates_mean

        # 랜덤으로 영화 선택
        res = movies_df.sample(cnt)

        # 영화별 별점 평균값 df 생성
        rates_mean_df = pd.DataFrame(rates_df.groupby('movie')['rate'].mean())
        rates_mean_df.columns = ['rates_mean']
        res = pd.merge(res, rates_mean_df, left_on='movie', right_index=True, how='left')
        res['rates_mean'] = res['rates_mean'].fillna(rates_mean)
        res['rate_random'] = np.random.uniform(1, 10, size=cnt).round(3)
        res['rate_random_class'] = np.round(res['rate_random'], 0).astype(int)
        res = res.sort_values(by='rate_random', ascending=False)
        return res.reset_index()
        

> Analyzer

- 결과를 분석하는 class
  - 평균 절대 오차(MAE)
  - 평균 제곱 오차(MSE)
  - 평균 제곱근 오차(RSME)
  - 평균 절대 비율 오차(MAPE)

In [134]:
def create_confusion_matrix(y_true, y_pred):
    """sklearn 없이 혼동 행렬 생성"""
    if len(y_true) != len(y_pred):
        raise ValueError("y_true와 y_pred의 길이가 같아야 합니다.")

    n_labels = 10
    cm = np.zeros((n_labels, n_labels), dtype=int)         

    for true_label, pred_label in zip(y_true, y_pred):
        if not isinstance(true_label, np.ndarray):
            continue
        for index, cnt in enumerate(true_label):
            cm[index][pred_label - 1] += cnt
    return cm



In [135]:
def get_freq_list(x):
    res = np.zeros(10)
    for item in x:
        res[item - 1] += 1
    return res

class Analyzer():
    def __init__(self):
        self.dfs = {}
        self.dfs['rates'] = MovieDateLoader('rates')
        self.dfs['movies'] = MovieDateLoader('movies')

    def mae(self, df: pd.DataFrame, real: str, pred: str):
        cnt = len(df)
        return np.sum(np.abs(df[real] - df[pred])) / cnt
        
    def mse(self, df: pd.DataFrame, real: str, pred: str):
        cnt = len(df)
        return np.sum(np.power(df[real] - df[pred], 2)) / cnt
    
    def rmse(self, df: pd.DataFrame, real: str, pred: str):
        return np.sqrt(self.mse(df, real, pred))
    
    def mape(self, df: pd.DataFrame, real: str, pred: str):
        cnt = len(df)
        return (np.sum(np.abs(df[real] - df[pred]) / np.abs(df[real])) / cnt) * 100
        
    def analyze(self, df: pd.DataFrame, real: str, pred: str):
        print(f"MAE: {self.mae(df, real, pred)}")
        print(f"MSE: {self.mse(df, real, pred)}")
        print(f"RMSE: {self.rmse(df, real, pred)}")
        print(f"MAPE: {self.mape(df, real, pred)}")

    def confusion_items(self, df: pd.DataFrame):
        tp = np.diag(df)
        fp = np.sum(df, axis=0) - tp # 세로합 
        tn = 0
        fn = np.sum(df, axis=1) - tp # 가로합
        return {
            'tp': tp, 'fp': fp, 'tn': tn, 'fn': fn
        }

    def analyze_classify(self, df: pd.DataFrame):
        rates_df = self.dfs['rates'].df
        rates_classify = rates_df.groupby('movie')['rate'].apply(list).apply(get_freq_list)
        res = pd.merge(df, rates_classify, how='left', left_on='movie', right_index=True)
        confus_mat = create_confusion_matrix(res['rate'], res['rate_random_class'])
        print(confus_mat)
        items = self.confusion_items(confus_mat)
        accu = (items['tp'].sum() + items['tn']) / (items['tp'].sum() + items['tn'] + items['fp'].sum() + items['fn'].sum())
        prec = items['tp'] / (items['tp'] + items['fp'])
        recall = items['tp'] / (items['tp'] + items['fn'])
        print(f"Accuracy : {accu}")
        print(f"Precisioon :{prec}")
        print(f"Recall :{recall}")
        print(f"F1_Score :{2 * prec * recall / (prec + recall)}")



In [136]:
recommender = RandomRecommender()
analyzer = Analyzer()
cnt = get_integer('몇개의 영화를 추천드릴까요?')
try:
    recommends = recommender.run(cnt)

    print(recommends)
    analyzer.analyze(recommends, 'rates_mean', 'rate_random')

    analyzer.analyze_classify(recommends)
except:
    print(f"Error occured")



     index  movie        title  \
0      357  10358         애니 홀   
1      169  10170  007 뷰 투 어 킬   
2      750  10751        블루 라군   
3       12  10013           킹콩   
4      677  10678         종합병원   
..     ...    ...          ...   
994    504  10505         은행풍운   
995    335  10336    암흑가의 두 사람   
996    111  10112       영웅본색 3   
997    827  10828      영구와 땡칠이   
998     51  10052        싸이코 3   

                                             title_eng    year     grade  \
0                                    Annie Hall , 1977     NaN        PG   
1                              A View To A Kill , 1985  1985.0        PG   
2                               The Blue Lagoon , 1980  1988.0  청소년 관람불가   
3                                     King Kong , 1976  1977.0   12세 관람가   
4                                  The Hospital , 1971     NaN        PG   
..                                                 ...     ...       ...   
994                         人民英雄: People's Hero , 1987     